In [36]:
import pathlib
import sys
import shutil
import tempfile
import os
import subprocess
import csv
import biom
import numpy as np
import pandas as pd

from qiime2 import Artifact, Metadata, MetadataColumn
from qiime2.plugins import cutadapt as cut
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.feature_table.methods import filter_seqs
from qiime2.plugins.feature_table.methods import filter_features
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize as summarize_table
from qiime2.plugins.diversity.pipelines import core_metrics
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_group_significance


from qiime2.plugins.feature_classifier.pipelines import classify_hybrid_vsearch_sklearn
from qiime2.plugins.feature_classifier.methods import classify_consensus_vsearch, classify_sklearn
from qiime2.plugins.taxa.visualizers import barplot
from qiime2.plugins.phylogeny.pipelines import align_to_tree_mafft_fasttree
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic



In [41]:
rep_seqs = Artifact.load('QIIME_Artifacts/rep_seqs_filt.qza')
table = Artifact.load('QIIME_Artifacts/table_filt.qza')
sample_map = 'Sample_Map_Parasite.txt'

metadata = Metadata.load(sample_map)
metadata_df = pd.read_csv(sample_map, sep='\t')
metadata_df

,#SampleID,BarcodeSequence,Sample_year,Sample_Name,MORPH,SEX,PAIR_TYPE,HABITAT
0,bc1001,TCAGACGATGCGTCAT,2019,GFGB,Tan,Male,TxW,Bog
1,bc1002,CTATACATGACTCTGC,2019,FWWW,Tan,Female,WxT,Bog
2,bc1003,TACTAGAGTAGCACTC,2019,PFGR,White,Female,TxW,Pond
3,bc1004,TGTGTATCAGTACATG,2019,WKOF,Tan,Male,TxW,Forest
4,bc1005,ACACGCATGACACACT,2019,RWWF,Tan,Female,WxT,Forest
...,...,...,...,...,...,...,...,...
86,bc1091,CACACGAGATCTCATC,2007,FORW.2,Tan,Male,TxW,Forest
87,bc1092,AGATACACATGATACT,2013,GFOO,White,Male,WxT,Bog
88,bc1094,TCTCGACTGCACATAT,2011,BOFG.2,Tan,Female,TxT,Bog
89,bc1095,TGAGTGACGTGTAGCG,2011,YFOW,Tan,Male,TxW,Pond


In [18]:
(summarize_table(table)).visualization

<visualization: Visualization uuid: 64dcb184-fc48-4b35-b62e-6839cb1ac276>

In [19]:
alignment, masked_alignment, unrooted_tree, rooted_tree = align_to_tree_mafft_fasttree(rep_seqs,
                                                                                      n_threads=35)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 35 /tmp/qiime2-archive-hjwl1cr8/e5f03fa5-9825-4dae-8086-879fc47284bc/data/dna-sequences.fasta



inputfile = orig
260 x 484 - 471 d
nthread = 35
nthreadpair = 35
nthreadtb = 16
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  201 / 260 (thread   17)
done.

Constructing a UPGMA tree (efffree=0) ... 
  250 / 260
done.

Progressive alignment 1/2... 
STEP    95 / 259 (thread   14) f
Reallocating..done. *alloclen = 2067


STEP   259 / 259 (thread   11) f
done.

Making a distance matrix from msa.. 
  200 / 260 (thread    4)
done.

Constructing a UPGMA tree (efffree=1) ... 
  250 / 260
done.

Progressive alignment 2/2... 
STEP   224 / 259 (thread    8) f
Reallocating..done. *alloclen = 2299


STEP   259 / 259 (thread    9) f
done.

disttbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
16 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.



Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTreeMP -quote -nt /tmp/qiime2-archive-7vejfflq/1ceaa665-3d12-41a8-bafb-eeb756f40597/data/aligned-dna-sequences.fasta



FastTree Version 2.1.10 Double precision (No SSE3), OpenMP (35 threads)
Alignment: /tmp/qiime2-archive-7vejfflq/1ceaa665-3d12-41a8-bafb-eeb756f40597/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.13 seconds: Joined    100 of    257
Initial topology in 0.21 seconds
Refining topology: 32 rounds ME-NNIs, 2 rounds ME-SPRs, 16 rounds ML-NNIs
      0.23 seconds: ME NNI round 3 of 32, 1 of 258 splits
      0.37 seconds: SPR round   1 of   2, 101 of 518 nodes
      0.49 seconds: SPR round   1 of   2, 201 of 518 nodes
      0.69 seconds: SPR round   1 of   2, 401 of 518 nodes
      0.79 seconds: ME NNI round 11 of 32, 1 of 258 splits
      0.94 seconds: SPR round   2 of   2, 101 of 518 nodes
      1.04 seconds: SPR round   2 of   2, 201 of 518 nodes
     

In [20]:
# Core metrics non-phylo @ 1000

rt, oov, sv, ev, jdm, bcdm, jpr, bcpa, je, bc = core_metrics(table, 1000, metadata, n_jobs=35)

/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.11332806904047718 and the largest is 5.017968917453181.
  warn(
/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude wit

In [21]:
# Core metrics phylo @ 1000
rt, fpdv, oov, sv, ev, uudm, wudm, jdm, bcdm, uupr, wupr, jpr, bcpr, uue, wue, je, bce = core_metrics_phylogenetic(table, 
                                                                                                      rooted_tree,
                                                                                                      1000, 
                                                                                                      metadata, 
                                                                                                      n_jobs_or_threads=35,
                                                                                                      )

/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.11065427485965561 and the largest is 5.222929602997439.
  warn(
/media/extra/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude wit

In [32]:
#PD Alpha
fpd_sig = alpha_group_significance(fpdv, metadata)
fpd_sig.visualization

<visualization: Visualization uuid: 78b541f1-18aa-4e9a-bc74-ffbd6a2dbd15>

In [44]:
#Unweighted Beta
habitat = Metadata.get_column(metadata, 'HABITAT')
sex = Metadata.get_column(metadata, 'SEX')
morph = Metadata.get_column(metadata, 'MORPH')
uudm_sig = beta_group_significance(uudm, morph)

In [45]:
uudm_sig.visualization

<visualization: Visualization uuid: 71ba55aa-c985-46e4-8622-3a9ef6f4d2e2>